In [6]:
import ccxt
import pandas as pd
from datetime import datetime

# Función para convertir fecha a milisegundos desde la época Unix
def fecha_a_milisegundos(fecha):
    return int(datetime.strptime(fecha, '%d-%m-%Y').timestamp() * 1000)

# Función para descargar datos históricos de un exchange
def descargar_datos_historicos(exchange, coin_id, desde, hasta):
    datos_historicos = []
    limite = 1000  # El número de puntos de datos por solicitud, ajustar según el exchange

    while desde < hasta:
        try:
            ohlcv = exchange.fetch_ohlcv(coin_id, '1d', desde, limite)
            datos_historicos.extend(ohlcv)
            if len(ohlcv) == 0:
                break
            desde = ohlcv[-1][0] + 1
        except Exception as e:
            print(f"Error al descargar datos para {coin_id}: {e}")
            break

    return datos_historicos

# Configuración inicial
exchange = ccxt.binance()  # Puedes cambiar a cualquier exchange soportado por ccxt

# Lista de las 10 principales criptomonedas
criptos_interesantes = [
    'BTC/USDT',  # Bitcoin
    'ETH/USDT',  # Ethereum
    'BNB/USDT',  # Binance Coin
    'XRP/USDT',  # XRP
    'SOL/USDT',  # Solana
    'ADA/USDT',  # Cardano
    'DOT/USDT',  # Polkadot
    'DOGE/USDT', # Dogecoin
    'AVAX/USDT', # Avalanche
    'MATIC/USDT' # Polygon
]

desde = fecha_a_milisegundos('01-01-2010')
hasta = exchange.milliseconds()

# DataFrame para consolidar los resultados
df_consolidado = pd.DataFrame()

for coin_symbol in criptos_interesantes:
    try:
        print(f"Descargando datos para {coin_symbol}...")
        datos = descargar_datos_historicos(exchange, coin_symbol, desde, hasta)

        if datos:
            df = pd.DataFrame(datos, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df['symbol'] = coin_symbol

            # SMA - Media Móvil Simple
            df['sma_50'] = df['close'].rolling(window=50).mean()
            
            # EMA - Media Móvil Exponencial
            df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
            
            # RSI - Índice de Fuerza Relativa
            delta = df['close'].diff(1)
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)
            avg_gain = gain.rolling(window=14).mean()
            avg_loss = loss.rolling(window=14).mean()
            rs = avg_gain / avg_loss
            df['rsi'] = 100 - (100 / (1 + rs))
            
            # MACD - Convergencia y Divergencia de Medias Móviles
            df['ema_12'] = df['close'].ewm(span=12, adjust=False).mean()
            df['ema_26'] = df['close'].ewm(span=26, adjust=False).mean()
            df['macd'] = df['ema_12'] - df['ema_26']
            df['signal_line'] = df['macd'].ewm(span=9, adjust=False).mean()

            # Banda de Bollinger
            df['bollinger_upper'] = df['sma_50'] + 2 * df['close'].rolling(window=50).std()
            df['bollinger_lower'] = df['sma_50'] - 2 * df['close'].rolling(window=50).std()

            # Tasa de Cambio (ROC)
            df['roc'] = df['close'].pct_change(periods=9) * 100

            # Identificar picos históricos
            df['top_3_highs'] = df['high'].nlargest(3)
            df['bottom_3_lows'] = df['low'].nsmallest(3)

            # Concatenar con el DataFrame consolidado
            df_consolidado = pd.concat([df_consolidado, df])

        else:
            print(f"No se encontraron datos para {coin_symbol} o hubo un error.")
    except Exception as e:
        print(f"Error general al procesar {coin_symbol}: {e}")

# Guardar DataFrame consolidado en un archivo CSV
    nombre_archivo_consolidado = "criptomonedas_historico_consolidado.csv"
    df_consolidado.to_csv(nombre_archivo_consolidado, index=False)
    print(f"Todos los datos consolidados guardados en '{nombre_archivo_consolidado}'")

print("Proceso completado.")


Descargando datos para BTC/USDT...
Todos los datos consolidados guardados en 'criptomonedas_historico_consolidado.csv'
Descargando datos para ETH/USDT...
Todos los datos consolidados guardados en 'criptomonedas_historico_consolidado.csv'
Descargando datos para BNB/USDT...
Todos los datos consolidados guardados en 'criptomonedas_historico_consolidado.csv'
Descargando datos para XRP/USDT...
Todos los datos consolidados guardados en 'criptomonedas_historico_consolidado.csv'
Descargando datos para SOL/USDT...
Todos los datos consolidados guardados en 'criptomonedas_historico_consolidado.csv'
Descargando datos para ADA/USDT...
Todos los datos consolidados guardados en 'criptomonedas_historico_consolidado.csv'
Descargando datos para DOT/USDT...
Todos los datos consolidados guardados en 'criptomonedas_historico_consolidado.csv'
Descargando datos para DOGE/USDT...
Todos los datos consolidados guardados en 'criptomonedas_historico_consolidado.csv'
Descargando datos para AVAX/USDT...
Todos los d

In [7]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta

# Función para convertir fecha a milisegundos desde la época Unix
def fecha_a_milisegundos(fecha):
    return int(datetime.strptime(fecha, '%d-%m-%Y').timestamp() * 1000)

# Función para descargar datos históricos de un exchange
def descargar_datos_historicos(exchange, coin_id, desde, hasta):
    datos_historicos = []
    limite = 1000  # El número de puntos de datos por solicitud, ajustar según el exchange

    while desde < hasta:
        try:
            ohlcv = exchange.fetch_ohlcv(coin_id, '1d', desde, limite)
            datos_historicos.extend(ohlcv)
            if len(ohlcv) == 0:
                break
            desde = ohlcv[-1][0] + 1
        except Exception as e:
            print(f"Error al descargar datos para {coin_id}: {e}")
            break

    return datos_historicos

# Configuración inicial
exchange = ccxt.binance()  # Puedes cambiar a cualquier exchange soportado por ccxt

# Lista de las 10 principales criptomonedas
criptos_interesantes = criptos_para_pruebas = [
    'LTC/USDT',   # Litecoin
    'LINK/USDT',  # Chainlink
    'UNI/USDT',   # Uniswap
    'XTZ/USDT',   # Tezos
    'ALGO/USDT',  # Algorand
    'ATOM/USDT',  # Cosmos
    'VET/USDT',   # VeChain
    'AAVE/USDT',  # Aave
    'XLM/USDT',   # Stellar
    'ZEC/USDT'    # Zcash
]

# Calcula la fecha inicial retrocediendo una semana desde la fecha actual
fecha_actual = datetime.now()
fecha_inicial = fecha_actual - timedelta(weeks=1)
desde = fecha_a_milisegundos(fecha_inicial.strftime('%d-%m-%Y'))
hasta = exchange.milliseconds()

# DataFrame para consolidar los resultados
df_consolidado = pd.DataFrame()

for coin_symbol in criptos_interesantes:
    try:
        print(f"Descargando datos para {coin_symbol}...")
        datos = descargar_datos_historicos(exchange, coin_symbol, desde, hasta)

        if datos:
            df = pd.DataFrame(datos, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df['symbol'] = coin_symbol

            # SMA - Media Móvil Simple
            df['sma_50'] = df['close'].rolling(window=50).mean()
            
            # EMA - Media Móvil Exponencial
            df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
            
            # RSI - Índice de Fuerza Relativa
            delta = df['close'].diff(1)
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)
            avg_gain = gain.rolling(window=14).mean()
            avg_loss = loss.rolling(window=14).mean()
            rs = avg_gain / avg_loss
            df['rsi'] = 100 - (100 / (1 + rs))
            
            # MACD - Convergencia y Divergencia de Medias Móviles
            df['ema_12'] = df['close'].ewm(span=12, adjust=False).mean()
            df['ema_26'] = df['close'].ewm(span=26, adjust=False).mean()
            df['macd'] = df['ema_12'] - df['ema_26']
            df['signal_line'] = df['macd'].ewm(span=9, adjust=False).mean()

            # Banda de Bollinger
            df['bollinger_upper'] = df['sma_50'] + 2 * df['close'].rolling(window=50).std()
            df['bollinger_lower'] = df['sma_50'] - 2 * df['close'].rolling(window=50).std()

            # Tasa de Cambio (ROC)
            df['roc'] = df['close'].pct_change(periods=9) * 100

            # Identificar picos históricos
            df['top_3_highs'] = df['high'].nlargest(3)
            df['bottom_3_lows'] = df['low'].nsmallest(3)

            # Concatenar con el DataFrame consolidado
            df_consolidado = pd.concat([df_consolidado, df])

        else:
            print(f"No se encontraron datos para {coin_symbol} o hubo un error.")
    except Exception as e:
        print(f"Error general al procesar {coin_symbol}: {e}")

# Guardar los datos consolidados en un archivo CSV
nombre_archivo_consolidado = "criptomonedas_historico_consolidado_semana.csv"
df_consolidado.to_csv(nombre_archivo_consolidado, index=False)
print(f"Todos los datos consolidados guardados en '{nombre_archivo_consolidado}'")

print("Proceso completado.")



Descargando datos para LTC/USDT...
Descargando datos para LINK/USDT...
Descargando datos para UNI/USDT...
Descargando datos para XTZ/USDT...
Descargando datos para ALGO/USDT...
Descargando datos para ATOM/USDT...
Descargando datos para VET/USDT...
Descargando datos para AAVE/USDT...
Descargando datos para XLM/USDT...
Descargando datos para ZEC/USDT...
Todos los datos consolidados guardados en 'criptomonedas_historico_consolidado_semana.csv'
Proceso completado.


In [8]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer, mean_squared_error, r2_score

# Cargar datos
df = pd.read_csv("criptomonedas_historico_consolidado.csv")

# Preprocesamiento de datos
features = ['open', 'high', 'low', 'volume', 'sma_50', 'ema_20', 'rsi', 'macd', 'bollinger_upper', 'bollinger_lower', 'roc']
X = df[features]
y = df['close']

# Imputar los NaNs
imputador = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputador.fit_transform(X))
X_imputed.columns = X.columns

# Definir el modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Configurar la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Definir métricas personalizadas
def mse_score(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

def r2_score_custom(y_true, y_pred):
    return r2_score(y_true, y_pred)

mse_scorer = make_scorer(mse_score, greater_is_better=False)
r2_scorer = make_scorer(r2_score_custom, greater_is_better=True)

# Calcular la puntuación de validación cruzada para MSE y R²
mse_scores = cross_val_score(model, X_imputed, y, scoring=mse_scorer, cv=kf)
r2_scores = cross_val_score(model, X_imputed, y, scoring=r2_scorer, cv=kf)

# Convertir las puntuaciones MSE a positivo
mse_scores = -mse_scores

print("MSE scores:", mse_scores)
print("Mean MSE:", mse_scores.mean())
print("R² scores:", r2_scores)
print("Mean R²:", r2_scores.mean())


MSE scores: [39106.06527197 34236.42541511 32681.45864656 40489.51754971
 36235.32339558]
Mean MSE: 36549.7580557865
R² scores: [0.99954105 0.9995661  0.99959146 0.99947247 0.99952235]
Mean R²: 0.9995386865832459


In [9]:

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

# Cargar datos de entrenamiento
df_train = pd.read_csv("criptomonedas_historico_consolidado.csv")

# Cargar datos de prueba
df_test = pd.read_csv("criptomonedas_historico_consolidado_semana.csv")

# Preprocesamiento de datos para el entrenamiento
features = ['open', 'high', 'low', 'volume', 'sma_50', 'ema_20', 'rsi', 'macd', 'bollinger_upper', 'bollinger_lower', 'roc']
X_train = df_train[features]
y_train = df_train['close']

# Imputar los NaNs para el conjunto de entrenamiento
imputador = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputador.fit_transform(X_train))
X_train_imputed.columns = X_train.columns

# Preprocesamiento de datos para la prueba
X_test = df_test[features]
y_test = df_test['close']

# Imputar los NaNs para el conjunto de prueba
X_test_imputed = pd.DataFrame(imputador.transform(X_test))
X_test_imputed.columns = X_test.columns

# Definir el modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_imputed, y_train)

# Predecir con los datos de prueba
y_pred = model.predict(X_test_imputed)

# Calcular métricas de rendimiento
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE) en los datos de prueba:", mse)
print("Coeficiente de Determinación (R²) en los datos de prueba:", r2)




Mean Squared Error (MSE) en los datos de prueba: 0.9800658145617851
Coeficiente de Determinación (R²) en los datos de prueba: 0.9989598176904364


In [11]:
# Crear un DataFrame con las predicciones
df_predicciones = pd.DataFrame(y_pred, columns=['close_pred'])

# Añadir la columna de predicciones al DataFrame de prueba
# Asegúrate de resetear el índice si es necesario
df_test.reset_index(drop=True, inplace=True)
df_resultados = pd.concat([df_test, df_predicciones], axis=1)

# Ahora df_resultados contiene tanto los datos reales como las predicciones
df_resultados


,timestamp,open,high,low,close,volume,symbol,sma_50,ema_20,rsi,ema_12,ema_26,macd,signal_line,bollinger_upper,bollinger_lower,roc,top_3_highs,bottom_3_lows,close_pred
0,2023-11-16,74.02,74.35,69.78,70.97,516234.012,LTC/USDT,NaN,70.970000,NaN,70.970000,70.970000,0.000000,0.000000,NaN,NaN,NaN,74.35,NaN,70.226754
1,2023-11-17,70.96,71.66,68.00,70.29,701059.335,LTC/USDT,NaN,70.905238,NaN,70.865385,70.919630,-0.054245,-0.010849,NaN,NaN,NaN,71.66,NaN,70.230402
2,2023-11-18,70.28,70.33,67.50,69.86,319967.121,LTC/USDT,NaN,70.805692,NaN,70.710710,70.841139,-0.130428,-0.034765,NaN,NaN,NaN,NaN,67.50,68.575972
3,2023-11-19,69.85,70.67,68.50,70.54,267656.397,LTC/USDT,NaN,70.780388,NaN,70.684447,70.818832,-0.134385,-0.054689,NaN,NaN,NaN,NaN,NaN,70.202630
4,2023-11-20,70.56,71.02,68.62,69.51,363216.736,LTC/USDT,NaN,70.659398,NaN,70.503763,70.721881,-0.218119,-0.087375,NaN,NaN,NaN,NaN,NaN,70.270472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2023-11-19,28.89,29.36,28.17,29.23,90336.257,ZEC/USDT,NaN,29.438095,NaN,29.384561,29.459119,-0.074558,-0.027422,NaN,NaN,NaN,NaN,NaN,28.819462
76,2023-11-20,29.25,29.64,28.40,28.92,108151.209,ZEC/USDT,NaN,29.388753,NaN,29.313090,29.419184,-0.106094,-0.043156,NaN,NaN,NaN,29.64,NaN,28.857817
77,2023-11-21,28.96,29.33,26.53,26.80,145364.209,ZEC/USDT,NaN,29.142205,NaN,28.926461,29.225171,-0.298710,-0.094267,NaN,NaN,NaN,NaN,26.53,27.768402
78,2023-11-22,26.78,28.72,26.72,28.50,145593.404,ZEC/USDT,NaN,29.081043,NaN,28.860851,29.171454,-0.310603,-0.137534,NaN,NaN,NaN,NaN,26.72,28.150379
